In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import classification_report
import numpy as np

In [5]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# -----------------------------
# Environment Setup
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = (1, 28, 28)  # Grayscale images
num_classes = 10  # Example: 10-class classification

# -----------------------------
# Preprocessing
# -----------------------------
num_samples = 1000
X = np.random.rand(num_samples, *input_shape).astype(np.float32)  # Random pixel values between 0-1
y = np.random.randint(0, num_classes, size=(num_samples,))

# Convert to PyTorch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)

# -----------------------------
# Train-Test Split
# -----------------------------
train_size = int(0.8 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [6]:
# Train Base Model (CNN)
# -----------------------------
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 7 * 7, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

model = CNNModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, epochs=2):
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                val_outputs = model(X_val)
                val_loss += criterion(val_outputs, y_val).item()
        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

train_model(model, train_loader, val_loader)

Epoch 1/2, Validation Loss: 2.3290
Epoch 2/2, Validation Loss: 2.3282


In [7]:
# Planning (Simulated Experience Using CNN Predictions)
# -----------------------------
simulated_data = torch.tensor(np.random.rand(50, *input_shape).astype(np.float32)).to(device)
model.eval()
with torch.no_grad():
    simulated_predictions = model(simulated_data)

# -----------------------------
# Fine-Tune Model
# -----------------------------
class FineTunedCNN(CNNModel):
    def __init__(self, num_classes):
        super(FineTunedCNN, self).__init__(num_classes)
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 7 * 7, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

fine_tuned_model = FineTunedCNN(num_classes).to(device)
optimizer = optim.Adam(fine_tuned_model.parameters(), lr=5e-4)

train_model(fine_tuned_model, train_loader, val_loader, epochs=2)

# -----------------------------
# Evaluate
# -----------------------------
all_preds, all_labels = [], []
fine_tuned_model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = fine_tuned_model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

# -----------------------------
# Deploy Policy (Simulated)
# -----------------------------
torch.save(fine_tuned_model.state_dict(), 'cnn_model.pth')
print("\nModel saved as 'cnn_model.pth'")

Epoch 1/2, Validation Loss: 2.3277
Epoch 2/2, Validation Loss: 2.3220

Classification Report:
              precision    recall  f1-score   support

           0       0.06      1.00      0.11         6
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        12
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00         5

    accuracy                           0.06       100
   macro avg       0.01      0.10      0.01       100
weighted avg       0.00      0.06      0.01       100


Model saved as 'cnn_model.pth'
